In [27]:
import pandas as pd
import numpy as np

def calculate_distance_matrix(df) -> pd.DataFrame():
  unique_ids = pd.unique(df[['id_start', 'id_end']].values.ravel())
  distance_matrix = pd.DataFrame(np.inf, index=unique_ids, columns=unique_ids)
  np.fill_diagonal(distance_matrix.values, 0)
  for _, row in df.iterrows():
    start, end, distance = row['id_start'], row['id_end'], row['distance']
    distance_matrix.at[start, end] = distance
    distance_matrix.at[end, start] = distance
  for k in unique_ids:
    for i in unique_ids:
      for j in unique_ids:
        distance_matrix.at[i, j] = min(distance_matrix.at[i, j], distance_matrix.at[i, k] + distance_matrix.at[k, j])

  return distance_matrix


In [28]:
file_path = '/content/dataset-2.csv'
df = pd.read_csv(file_path)
distance_matrix = calculate_distance_matrix(df)
print(distance_matrix)


         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      0.0      9.7     29.9     45.9     67.6     78.7     94.3   
1001402      9.7      0.0     20.2     36.2     57.9     69.0     84.6   
1001404     29.9     20.2      0.0     16.0     37.7     48.8     64.4   
1001406     45.9     36.2     16.0      0.0     21.7     32.8     48.4   
1001408     67.6     57.9     37.7     21.7      0.0     11.1     26.7   
1001410     78.7     69.0     48.8     32.8     11.1      0.0     15.6   
1001412     94.3     84.6     64.4     48.4     26.7     15.6      0.0   
1001414    112.5    102.8     82.6     66.6     44.9     33.8     18.2   
1001416    125.7    116.0     95.8     79.8     58.1     47.0     31.4   
1001418    139.3    129.6    109.4     93.4     71.7     60.6     45.0   
1001420    152.2    142.5    122.3    106.3     84.6     73.5     57.9   
1001422    161.8    152.1    131.9    115.9     94.2     83.1     67.5   
1001424    173.2    163.5    143.3    

In [29]:
import pandas as pd
def unroll_distance_matrix(df) -> pd.DataFrame():
  unrolled_data = []

  for id_start in df.index:
    for id_end in df.columns:
      if id_start != id_end:
        distance = df.at[id_start, id_end]
        unrolled_data.append({'id_start': id_start, 'id_end': id_end, 'distance': distance})

  unrolled_df = pd.DataFrame(unrolled_data)
  return unrolled_df

In [30]:
distance_data = pd.read_csv('/content/dataset-2.csv')
unrolled_df = unroll_distance_matrix(distance_matrix)
print(unrolled_df)

      id_start   id_end  distance
0      1001400  1001402       9.7
1      1001400  1001404      29.9
2      1001400  1001406      45.9
3      1001400  1001408      67.6
4      1001400  1001410      78.7
...        ...      ...       ...
1801   1001472  1001462      72.5
1802   1001472  1001464      45.8
1803   1001472  1001466      37.3
1804   1001472  1001468      26.6
1805   1001472  1001470      16.0

[1806 rows x 3 columns]


In [31]:
def find_ids_within_ten_percentage_threshold(df, reference_id) -> pd.DataFrame():
  reference_avg_distance = df[df['id_start'] == reference_id]['distance'].mean()
  lower_bound = reference_avg_distance * 0.9
  upper_bound = reference_avg_distance * 1.1

  avg_distances = df.groupby('id_start')['distance'].mean().reset_index()
  within_threshold = avg_distances[
    (avg_distances['distance'] >= lower_bound) & (avg_distances['distance'] <= upper_bound)
    ]
  within_threshold_sorted = within_threshold.sort_values(by='id_start').reset_index(drop=True)
  return within_threshold_sorted

In [32]:
reference_id = 1001400
result_df = find_ids_within_ten_percentage_threshold(unrolled_df, reference_id)
print(result_df)

   id_start    distance
0   1001400  243.995238
1   1001402  234.526190


In [33]:
import pandas as pd
def calculate_toll_rate(df: pd.DataFrame) -> pd.DataFrame:
    df['moto'] = df['distance'] * 0.8
    df['car'] = df['distance'] * 1.2
    df['rv'] = df['distance'] * 1.5
    df['bus'] = df['distance'] * 2.2
    df['truck'] = df['distance'] * 3.6
    return df

unrolled_df_with_toll = calculate_toll_rate(unrolled_df)
print(unrolled_df_with_toll)


      id_start   id_end  distance   moto    car      rv     bus   truck
0      1001400  1001402       9.7   7.76  11.64   14.55   21.34   34.92
1      1001400  1001404      29.9  23.92  35.88   44.85   65.78  107.64
2      1001400  1001406      45.9  36.72  55.08   68.85  100.98  165.24
3      1001400  1001408      67.6  54.08  81.12  101.40  148.72  243.36
4      1001400  1001410      78.7  62.96  94.44  118.05  173.14  283.32
...        ...      ...       ...    ...    ...     ...     ...     ...
1801   1001472  1001462      72.5  58.00  87.00  108.75  159.50  261.00
1802   1001472  1001464      45.8  36.64  54.96   68.70  100.76  164.88
1803   1001472  1001466      37.3  29.84  44.76   55.95   82.06  134.28
1804   1001472  1001468      26.6  21.28  31.92   39.90   58.52   95.76
1805   1001472  1001470      16.0  12.80  19.20   24.00   35.20   57.60

[1806 rows x 8 columns]


In [35]:
import pandas as pd
import datetime

def calculate_time_based_toll_rates(df: pd.DataFrame) -> pd.DataFrame:
    weekday_intervals = [
        (datetime.time(0, 0), datetime.time(10, 0), 0.8),
        (datetime.time(10, 0), datetime.time(18, 0), 1.2),
        (datetime.time(18, 0), datetime.time(23, 59, 59), 0.8)
    ]

    weekend_intervals = [
        (datetime.time(0, 0), datetime.time(23, 59, 59), 0.7)
    ]

    weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    weekends = ["Saturday", "Sunday"]

    rows = []

    for _, row in df.iterrows():
        for day in weekdays + weekends:
            if day in weekdays:
                intervals = weekday_intervals
            else:
                intervals = weekend_intervals

            for start_time, end_time, discount in intervals:
                new_row = row.copy()
                new_row['start_day'] = day
                new_row['end_day'] = day
                new_row['start_time'] = start_time
                new_row['end_time'] = end_time

                new_row['moto'] = row['moto'] * discount
                new_row['car'] = row['car'] * discount
                new_row['rv'] = row['rv'] * discount
                new_row['bus'] = row['bus'] * discount
                new_row['truck'] = row['truck'] * discount

                rows.append(new_row)

    result_df = pd.DataFrame(rows)

    return result_df


In [36]:
time_based_df = calculate_time_based_toll_rates(unrolled_df_with_toll)

In [37]:
print(time_based_df)

       id_start     id_end  distance    moto     car     rv     bus   truck  \
0     1001400.0  1001402.0       9.7   6.208   9.312  11.64  17.072  27.936   
0     1001400.0  1001402.0       9.7   9.312  13.968  17.46  25.608  41.904   
0     1001400.0  1001402.0       9.7   6.208   9.312  11.64  17.072  27.936   
0     1001400.0  1001402.0       9.7   6.208   9.312  11.64  17.072  27.936   
0     1001400.0  1001402.0       9.7   9.312  13.968  17.46  25.608  41.904   
...         ...        ...       ...     ...     ...    ...     ...     ...   
1805  1001472.0  1001470.0      16.0  10.240  15.360  19.20  28.160  46.080   
1805  1001472.0  1001470.0      16.0  15.360  23.040  28.80  42.240  69.120   
1805  1001472.0  1001470.0      16.0  10.240  15.360  19.20  28.160  46.080   
1805  1001472.0  1001470.0      16.0   8.960  13.440  16.80  24.640  40.320   
1805  1001472.0  1001470.0      16.0   8.960  13.440  16.80  24.640  40.320   

     start_day   end_day start_time  end_time  
0  